# HRF Convolution Pipeline for Cochlea and BEZ Models

This notebook provides an interactive interface to run HRF convolution on PSTH data from both Cochlea and BEZ models.

## Features
- Load PSTH data from either or both models
- **BEZ**: Process each simulation run separately (preserves run-to-run variability)
- **Cochlea**: Single deterministic response per condition
- Configure convolution parameters interactively
- Visualize canonical HRF
- Run convolution and inspect results
- Save convolved responses for later analysis
- Compare model outputs

## Workflow
1. Set up environment and parameters
2. Load model data
3. Generate and visualize canonical HRF
4. Run convolution
5. Visualize and save results

In [1]:
# Import required libraries
import sys
from pathlib import Path
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os
from IPython.display import display, HTML

# Set working directory
notebook_dir = Path.cwd()
if notebook_dir.name != 'convolution_HRF':
    convolution_hrf_dir = (
        notebook_dir / 'convolution_HRF' 
        if (notebook_dir / 'convolution_HRF').exists() 
        else notebook_dir.parent / 'convolution_HRF'
    )
    os.chdir(convolution_hrf_dir)
    print(f"Changed working directory to: {convolution_hrf_dir}")
else:
    print(f"Working directory: {notebook_dir}")

# Add current directory to path
sys.path.insert(0, str(Path.cwd()))

# Import convolution functions
# For Cochlea - uses standard method
from convolutionHRF_draft_221025 import (
    load_cochlea_psth_data,
    convolve_cochlea_with_hrf,
    plot_convolved_responses,
    subset_parameters
)

# For BEZ - uses per-run convolution method
from convolutionHRF_bez_eachrun_231025 import (
    load_bez_psth_data,
    convolve_bez_runs_with_hrf,
    plot_convolved_runs,
    plot_runs_comparison_across_cfs
)

# Utility functions
from convolution_utils import (
    generate_canonical_hrf,
    plot_hrf,
    generate_timestamp,
    save_convolved_data
)

print("✓ All modules imported successfully")
print("\nImportant: BEZ model uses per-run convolution to preserve variability")

Working directory: /home/ekim/PycharmProjects/phd_firstyear/subcorticalSTRF/convolution_HRF
✓ All modules imported successfully

Important: BEZ model uses per-run convolution to preserve variability
✓ All modules imported successfully

Important: BEZ model uses per-run convolution to preserve variability


## Step 1: Configuration

Set parameters for the convolution analysis. You can modify these settings as needed.

**Note on BEZ Model:** Each simulation run is convolved separately to preserve run-to-run variability, which is important for statistical analysis.

In [5]:
# === CONFIGURATION PARAMETERS ===

# Model selection
MODEL = 'both'  # Options: 'cochlea', 'bez', 'both'

# HRF parameters
TR = 0.1  # Temporal resolution in seconds (100 Hz sampling)
HRF_LENGTH = 32.0  # HRF duration in seconds

# Fiber types to process
FIBER_TYPES = ['hsr', 'msr', 'lsr']  # All fiber types

# Data subset parameters (set to None to process all)
MAX_CFS = None  # Maximum number of CFs to process
MAX_FREQS = None  # Maximum number of frequencies
MAX_DBS = None  # Maximum number of dB levels

# Test mode (process only small subset for quick testing)
TEST_MODE = True  # Set to True for quick testing

# Plotting parameters
DB_LEVEL_TO_PLOT = 60.0  # dB level for visualization
PLOT_FORMAT = 'png'  # Options: 'png', 'pdf', 'svg'

# Output settings
OUTPUT_DIR = 'results'
SAVE_PLOTS = True
SAVE_DATA = True

# Display configuration
print("=== Configuration Summary ===")
print(f"Model: {MODEL}")
print(f"TR: {TR}s (sampling rate: {1/TR:.0f} Hz)")
print(f"HRF length: {HRF_LENGTH}s")
print(f"Fiber types: {FIBER_TYPES}")
print(f"Test mode: {TEST_MODE}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Save plots: {SAVE_PLOTS}")
print(f"Save data: {SAVE_DATA}")

=== Configuration Summary ===
Model: both
TR: 0.1s (sampling rate: 10 Hz)
HRF length: 32.0s
Fiber types: ['hsr', 'msr', 'lsr']
Test mode: True
Output directory: results
Save plots: True
Save data: True


## Step 2: Load Model Data

Load PSTH data from the selected model(s).

**Data Structure:**
- **Cochlea**: Single response per condition `[fiber_type][CF][frequency][dB]`
- **BEZ**: Multiple runs per condition `[fiber_type][CF][frequency][dB][run_idx]`

In [6]:
# Load data based on model selection
print(f"=== Loading {MODEL.upper()} Model Data ===\n")

if MODEL == 'cochlea':
    model_data, full_parameters = load_cochlea_psth_data()
    
elif MODEL == 'bez':
    # Use the per-run BEZ loader
    model_data, full_parameters = load_bez_psth_data()
    
elif MODEL == 'both':
    print("Loading Cochlea data...")
    cochlea_data, cochlea_params = load_cochlea_psth_data()
    
    print("\nLoading BEZ data...")
    bez_data, bez_params = load_bez_psth_data()
    
    # Combine into single structure
    model_data = {
        'cochlea': cochlea_data,
        'bez': bez_data
    }
    
    # Parameters should be the same for both
    full_parameters = cochlea_params
    
    print("\n✓ Both models loaded successfully!")
else:
    raise ValueError(f"Invalid model: {MODEL}")

if MODEL != 'both':
    print("\n✓ Data loaded successfully!")
    print(f"\nFull parameter space:")
    print(f"  CFs: {len(full_parameters['cfs'])} values")
    print(f"  Frequencies: {len(full_parameters['frequencies'])} values")
    print(f"  dB levels: {full_parameters['dbs']}")
    
    if MODEL == 'bez':
        # Check number of runs
        sample_data = model_data['hsr_all']
        n_runs = sample_data.shape[3]
        print(f"  Runs per condition: {n_runs}")

=== Loading BOTH Model Data ===

Loading Cochlea data...
Loading cochlea PSTH data from: /home/ekim/PycharmProjects/phd_firstyear/subcorticalSTRF/cochlea_meanrate/out/condition_psths/cochlea_psths.mat
✓ Cochlea PSTH data loaded successfully
Parameters extracted:
  CFs: 20 values
  Frequencies: 20 values
  dB levels: [50. 60. 70. 80.]

Loading BEZ data...
Loading BEZ PSTH data from: //subcorticalSTRF/BEZ2018_meanrate/results/processed_data/psth_data_128fibers.mat


=== Loading BOTH Model Data ===

Loading Cochlea data...
Loading cochlea PSTH data from: /home/ekim/PycharmProjects/phd_firstyear/subcorticalSTRF/cochlea_meanrate/out/condition_psths/cochlea_psths.mat
✓ Cochlea PSTH data loaded successfully
Parameters extracted:
  CFs: 20 values
  Frequencies: 20 values
  dB levels: [50. 60. 70. 80.]

Loading BEZ data...
Loading BEZ PSTH data from: //subcorticalSTRF/BEZ2018_meanrate/results/processed_data/psth_data_128fibers.mat


FileNotFoundError: [Errno 2] No such file or directory: '//subcorticalSTRF/BEZ2018_meanrate/results/processed_data/psth_data_128fibers.mat'

## Step 3: Create Data Subset (Optional)

Create a subset of the full parameter space for faster processing during testing.

In [ ]:
# Create argument-like object for subset_parameters function
class SubsetArgs:
    def __init__(self):
        self.test = TEST_MODE
        self.max_cfs = MAX_CFS
        self.max_freqs = MAX_FREQS
        self.max_dbs = MAX_DBS

args = SubsetArgs()
parameters, indices = subset_parameters(full_parameters, args)

print("=== Data Subset Configuration ===")
print(f"Processing:")
print(f"  CFs: {len(parameters['cfs'])}/{len(full_parameters['cfs'])}")
print(f"  Frequencies: {len(parameters['frequencies'])}/{len(full_parameters['frequencies'])}")
print(f"  dB levels: {len(parameters['dbs'])}/{len(full_parameters['dbs'])}")
print(f"\nFirst few CFs: {parameters['cfs'][:5]}")
print(f"First few frequencies: {parameters['frequencies'][:5]}")
print(f"dB levels: {parameters['dbs']}")

## Step 4: Generate and Visualize Canonical HRF

Create the canonical hemodynamic response function that will be used for convolution.

In [ ]:
# Generate canonical HRF
print("=== Generating Canonical HRF ===\n")
hrf, time_points = generate_canonical_hrf(tr=TR, time_length=HRF_LENGTH)

print(f"HRF parameters:")
print(f"  Duration: {HRF_LENGTH}s")
print(f"  Sampling rate: {1/TR:.0f} Hz ({TR*1000:.1f} ms per sample)")
print(f"  Number of samples: {len(hrf)}")
print(f"  Peak time: ~{time_points[np.argmax(hrf)]:.1f}s")

# Plot the HRF
timestamp = generate_timestamp() if SAVE_PLOTS else None
plot_hrf(
    hrf, 
    time_points, 
    save_plots=SAVE_PLOTS,
    output_dir=OUTPUT_DIR,
    timestamp=timestamp,
    plot_format=PLOT_FORMAT
)

## Step 5: Run HRF Convolution

Convolve the PSTH data with the canonical HRF.

**Important Difference:**
- **Cochlea**: Convolves single response per condition
- **BEZ**: Convolves each simulation run separately (preserves variability)

**Note:** This step may take several minutes depending on the data size.

In [ ]:
# Run convolution based on model
print(f"=== Running HRF Convolution for {MODEL.upper()} ===\n")
print("This may take a few minutes...")

if MODEL == 'cochlea':
    # Standard convolution for Cochlea
    convolved_responses, hrf_used, time_axis = convolve_cochlea_with_hrf(
        model_data, 
        parameters, 
        indices, 
        FIBER_TYPES,
        tr=TR, 
        hrf_length=HRF_LENGTH
    )
    
elif MODEL == 'bez':
    # Per-run convolution for BEZ (preserves run variability)
    print("Processing each BEZ run separately...")
    convolved_responses, hrf_used, time_points = convolve_bez_runs_with_hrf(
        model_data,
        parameters,
        fiber_types=FIBER_TYPES,
        tr=TR,
        hrf_length=HRF_LENGTH
    )
    time_axis = time_points
    
elif MODEL == 'both':
    print("Processing Cochlea model...")
    cochlea_convolved, hrf_used, time_axis = convolve_cochlea_with_hrf(
        model_data['cochlea'],
        parameters,
        indices,
        FIBER_TYPES,
        tr=TR,
        hrf_length=HRF_LENGTH
    )
    
    print("\nProcessing BEZ model (each run separately)...")
    bez_convolved, _, _ = convolve_bez_runs_with_hrf(
        model_data['bez'],
        parameters,
        fiber_types=FIBER_TYPES,
        tr=TR,
        hrf_length=HRF_LENGTH
    )
    
    # Combine results
    convolved_responses = {
        'cochlea': cochlea_convolved,
        'bez': bez_convolved
    }

print("\n✓ Convolution complete!")
print(f"\nConvolved response structure:")
if MODEL == 'both':
    print(f"  Models: {list(convolved_responses.keys())}")
    print(f"  Cochlea fiber types: {list(convolved_responses['cochlea'].keys())}")
    print(f"  BEZ fiber types: {list(convolved_responses['bez'].keys())}")
elif MODEL == 'bez':
    print(f"  Fiber types: {list(convolved_responses.keys())}")
    print(f"  Structure: [fiber_type][CF][frequency][dB][run_idx]")
else:
    print(f"  Fiber types: {list(convolved_responses.keys())}")
    print(f"  Structure: [fiber_type][CF][frequency][dB]")

## Step 6: Visualize Convolved Responses

Plot the HRF-convolved responses for each fiber type.

**Plotting Options:**
- **Cochlea**: Shows single response trace per condition
- **BEZ**: Shows individual runs + mean ± SEM across runs

In [ ]:
# Plot convolved responses
print("=== Plotting Convolved Responses ===\n")

timestamp = generate_timestamp() if SAVE_PLOTS else None

if MODEL == 'cochlea':
    # Standard plotting for Cochlea
    for fiber_type in FIBER_TYPES:
        if fiber_type in convolved_responses:
            print(f"Plotting {fiber_type.upper()} fibers...")
            plot_convolved_responses(
                convolved_responses,
                parameters,
                fiber_type=fiber_type,
                db_val=DB_LEVEL_TO_PLOT,
                model_name='COCHLEA',
                save_plots=SAVE_PLOTS,
                output_dir=OUTPUT_DIR,
                timestamp=timestamp,
                plot_format=PLOT_FORMAT
            )

elif MODEL == 'bez':
    # Per-run plotting for BEZ
    for fiber_type in FIBER_TYPES:
        if fiber_type in convolved_responses:
            print(f"\nPlotting {fiber_type.upper()} fibers...")
            
            # Plot 1: Individual runs + mean/SEM for first CF
            print(f"  - Individual runs + mean/SEM...")
            plot_convolved_runs(
                convolved_responses,
                parameters,
                fiber_type=fiber_type,
                db_val=DB_LEVEL_TO_PLOT,
                tr=TR,
                plot_individual_runs=True,
                plot_mean_and_sem=True,
                max_runs_display=10,  # Show max 10 individual runs
                save_plots=SAVE_PLOTS,
                output_dir=OUTPUT_DIR,
                timestamp=timestamp,
                plot_format=PLOT_FORMAT
            )
            
            # Plot 2: Comparison across multiple CFs
            print(f"  - Comparison across CFs...")
            plot_runs_comparison_across_cfs(
                convolved_responses,
                parameters,
                fiber_type=fiber_type,
                db_val=DB_LEVEL_TO_PLOT,
                tr=TR,
                max_cfs=6,
                save_plots=SAVE_PLOTS,
                output_dir=OUTPUT_DIR,
                timestamp=timestamp,
                plot_format=PLOT_FORMAT
            )

elif MODEL == 'both':
    # Plot both models
    print("\nPlotting Cochlea model...")
    for fiber_type in FIBER_TYPES:
        if fiber_type in convolved_responses['cochlea']:
            plot_convolved_responses(
                convolved_responses['cochlea'],
                parameters,
                fiber_type=fiber_type,
                db_val=DB_LEVEL_TO_PLOT,
                model_name='COCHLEA',
                save_plots=SAVE_PLOTS,
                output_dir=OUTPUT_DIR,
                timestamp=timestamp,
                plot_format=PLOT_FORMAT
            )
    
    print("\nPlotting BEZ model (with runs)...")
    for fiber_type in FIBER_TYPES:
        if fiber_type in convolved_responses['bez']:
            plot_convolved_runs(
                convolved_responses['bez'],
                parameters,
                fiber_type=fiber_type,
                db_val=DB_LEVEL_TO_PLOT,
                tr=TR,
                save_plots=SAVE_PLOTS,
                output_dir=OUTPUT_DIR,
                timestamp=timestamp,
                plot_format=PLOT_FORMAT
            )

print("\n✓ Plotting complete!")

## Step 7: Save Convolved Data

Save the convolved responses to pickle files for later use.

**File Naming:**
- **Cochlea**: `COCHLEA_convolved_responses_{fiber_type}_{timestamp}.pkl`
- **BEZ**: `BEZ_convolved_responses_{fiber_type}_{timestamp}.pkl`

**Important:** BEZ files contain all runs separately for each condition.

In [ ]:
# Save convolved data
if SAVE_DATA:
    print("=== Saving Convolved Responses ===\n")
    
    timestamp = generate_timestamp()
    
    if MODEL == 'both':
        # Save each model separately
        print("Saving Cochlea results...")
        cochlea_output_dir = Path(OUTPUT_DIR) / 'cochlea'
        save_convolved_data(
            convolved_responses['cochlea'],
            str(cochlea_output_dir),
            timestamp,
            model_name='COCHLEA'
        )
        
        print("\nSaving BEZ results (with all runs)...")
        bez_output_dir = Path(OUTPUT_DIR) / 'bez'
        save_convolved_data(
            convolved_responses['bez'],
            str(bez_output_dir),
            timestamp,
            model_name='BEZ'
        )
        
        # Also save combined results
        combined_file = (
            Path(OUTPUT_DIR) / 
            f"combined_models_results_{timestamp}.pkl"
        )
        print(f"\nSaving combined results to: {combined_file}")
        with open(combined_file, 'wb') as f:
            pickle.dump(convolved_responses, f)
        print(f"✓ Combined data saved")
        
    else:
        # Save single model
        print(f"Saving {MODEL.upper()} results to: {OUTPUT_DIR}")
        save_convolved_data(
            convolved_responses,
            OUTPUT_DIR,
            timestamp,
            model_name=MODEL.upper()
        )
    
    print(f"\n✓ All data saved successfully!")
    print(f"Timestamp: {timestamp}")
else:
    print("Data saving skipped (SAVE_DATA=False)")

## Step 8: Inspect Results

Quick inspection of the convolved data structure and sample responses.

**Key Difference:**
- **Cochlea**: Single array per condition
- **BEZ**: Dictionary of arrays (one per run) per condition

In [ ]:
# Inspect convolved data structure
print("=== Data Structure Inspection ===\n")

if MODEL == 'both':
    # Inspect Cochlea
    print("COCHLEA Model:")
    for fiber_type in FIBER_TYPES:
        if fiber_type in convolved_responses['cochlea']:
            fiber_data = convolved_responses['cochlea'][fiber_type]
            n_cfs = len(fiber_data)
            sample_cf = list(fiber_data.keys())[0]
            n_freqs = len(fiber_data[sample_cf])
            sample_freq = list(fiber_data[sample_cf].keys())[0]
            n_dbs = len(fiber_data[sample_freq][sample_freq])
            sample_db = list(fiber_data[sample_freq][sample_freq].keys())[0]
            sample_response = fiber_data[sample_freq][sample_freq][sample_db]
            
            print(f"  {fiber_type.upper()}:")
            print(f"    CFs: {n_cfs}")
            print(f"    Frequencies per CF: {n_freqs}")
            print(f"    dB levels: {n_dbs}")
            print(f"    Sample response: {len(sample_response)} samples")
            print(f"    Duration: {len(sample_response) * TR:.1f}s")
    
    # Inspect BEZ
    print("\nBEZ Model (with runs):")
    for fiber_type in FIBER_TYPES:
        if fiber_type in convolved_responses['bez']:
            fiber_data = convolved_responses['bez'][fiber_type]
            n_cfs = len(fiber_data)
            sample_cf = list(fiber_data.keys())[0]
            n_freqs = len(fiber_data[sample_cf])
            sample_freq = list(fiber_data[sample_cf].keys())[0]
            n_dbs = len(fiber_data[sample_freq][sample_freq])
            sample_db = list(fiber_data[sample_freq][sample_freq].keys())[0]
            sample_runs = fiber_data[sample_freq][sample_freq][sample_db]
            
            # Count runs
            n_runs = len([r for r in sample_runs.values() if r is not None])
            sample_run = next(
                (r for r in sample_runs.values() if r is not None), 
                None
            )
            
            print(f"  {fiber_type.upper()}:")
            print(f"    CFs: {n_cfs}")
            print(f"    Frequencies per CF: {n_freqs}")
            print(f"    dB levels: {n_dbs}")
            print(f"    Runs per condition: {n_runs}")
            if sample_run is not None:
                print(f"    Sample run length: {len(sample_run)} samples")
                print(f"    Duration: {len(sample_run) * TR:.1f}s")

elif MODEL == 'bez':
    for fiber_type in FIBER_TYPES:
        if fiber_type in convolved_responses:
            fiber_data = convolved_responses[fiber_type]
            # ...existing inspection for BEZ...
            sample_cf = list(fiber_data.keys())[0]
            sample_freq = list(fiber_data[sample_cf].keys())[0]
            sample_db = list(fiber_data[sample_cf][sample_freq].keys())[0]
            sample_runs = fiber_data[sample_cf][sample_freq][sample_db]
            
            n_runs = len([r for r in sample_runs.values() if r is not None])
            sample_run = next(
                (r for r in sample_runs.values() if r is not None), 
                None
            )
            
            print(f"{fiber_type.upper()}:")
            print(f"  Structure: [CF][frequency][dB][run_idx]")
            print(f"  Runs per condition: {n_runs}")
            if sample_run is not None:
                print(f"  Sample run: {len(sample_run)} samples " 
                      f"({len(sample_run) * TR:.1f}s)")
else:
    # Cochlea only
    # ...existing code for cochlea...
    pass

## Summary

The HRF convolution analysis is complete! 

### What was done:
1. ✓ Loaded PSTH data from selected model(s)
2. ✓ Generated canonical HRF
3. ✓ Convolved PSTH data with HRF
   - **Cochlea**: Single response per condition
   - **BEZ**: Each simulation run processed separately
4. ✓ Visualized results
5. ✓ Saved convolved responses

### Key Differences Between Models:
- **Cochlea**: Deterministic model - single response per condition
- **BEZ**: Stochastic model - multiple runs per condition (preserves variability)

### Data Structure:
```
Cochlea: [fiber_type][CF][frequency][dB] → single_array
BEZ:     [fiber_type][CF][frequency][dB][run_idx] → run_array
```

### Next Steps:
- Extract specific dB levels using `test_extract_db_utility.ipynb`
- Run correlation analysis using `test_correlation_analysis.ipynb`
- Compare within-BEZ variability vs Cochlea-BEZ differences

### Files Generated:
Check the `results/` directory for:
- `BEZ_convolved_responses_{fiber_type}_{timestamp}.pkl` (with all runs)
- `COCHLEA_convolved_responses_{fiber_type}_{timestamp}.pkl`
- Plots (if enabled)

## Diagnostic: Check PSTH Temporal Resolution

Before running convolution, let's verify the actual temporal resolution of the PSTH data.

In [ ]:
# Search for clues about PSTH temporal resolution
import subprocess
from pathlib import Path

print("=== Searching for PSTH Temporal Resolution Clues ===\n")

# 1. Check MATLAB files for 'dt' or 'fs' or 'sampling' parameters
search_dir = Path.cwd().parent
print(f"Searching in: {search_dir}\n")

# Search patterns
patterns = [
    ("BEZ scripts", "BEZ*.py", ["dt", "fs", "sampling_rate", "time_step"]),
    ("Cochlea scripts", "*cochlea*.py", ["dt", "fs", "sampling"]),
    ("MATLAB files", "*.m", ["dt", "fs", "Fs"]),
]

for desc, file_pattern, search_terms in patterns:
    print(f"\n{desc} ({file_pattern}):")
    files = list(search_dir.rglob(file_pattern))[:5]  # First 5 matches
    
    for file in files:
        try:
            content = file.read_text()
            for term in search_terms:
                if term in content:
                    # Find lines containing the term
                    lines = [
                        line.strip() for line in content.split('\n') 
                        if term in line
                    ][:3]  # First 3 matches
                    if lines:
                        print(f"\n  {file.name}:")
                        for line in lines:
                            print(f"    {line}")
        except:
            continue

# 2. Check actual PSTH data structure
print("\n\n=== Checking Actual PSTH Data ===")

# Try loading a small sample
try:
    from scipy.io import loadmat
    
    # Try BEZ data
    bez_paths = [
        Path('/home/ekim/PycharmProjects/phd_firstyear/subcorticalSTRF/BEZ2018_meanrate/results/processed_data/psth_data_128fibers.mat'),
        Path('/home/ekim/audio_periph_models/BEZ2018a_model/results/processed_data/psth_data_128fibers.mat'),
    ]
    
    for bez_path in bez_paths:
        if bez_path.exists():
            print(f"\nLoading: {bez_path.name}")
            data = loadmat(str(bez_path), struct_as_record=False)
            
            # Check for timing information
            print("  Available fields:")
            for key in data.keys():
                if not key.startswith('__'):
                    print(f"    - {key}")
            
            # Check PSTH length
            if 'hsr_all' in data:
                sample = data['hsr_all']
                print(f"\n  PSTH data shape: {sample.shape}")
                print(f"  Likely dimensions: [n_CFs, n_freqs, n_dBs, n_runs]")
                
                # Get one PSTH
                psth_sample = sample[0, 0, 0, 0]
                if hasattr(psth_sample, 'flatten'):
                    psth_array = psth_sample.flatten()
                else:
                    psth_array = np.array(psth_sample)
                
                print(f"\n  Sample PSTH:")
                print(f"    Length: {len(psth_array)} samples")
                print(f"    If 200ms stimulus:")
                print(f"      @ 1000 Hz (1ms): {len(psth_array)} samples = {len(psth_array)}ms")
                print(f"      @ 10000 Hz (0.1ms): {len(psth_array)} samples = {len(psth_array)/10}ms")
                print(f"      @ 100 Hz (10ms): {len(psth_array)} samples = {len(psth_array)*10}ms")
            
            break
            
except Exception as e:
    print(f"Error loading data: {e}")

print("\n\n=== Conclusion ===")
print("Based on the PSTH length, determine which makes sense:")
print("  - 200 samples @ 1ms = 200ms stimulus ✓ (most likely)")
print("  - 2000 samples @ 0.1ms = 200ms stimulus")
print("  - 20 samples @ 10ms = 200ms stimulus")